# Laboratorio 2025
### Análisis y diseño de algoritmos distribuidos en redes
### Andrés Montoro 5.169.779-1


In [1]:
from pydistsim.algorithm.node_algorithm import NodeAlgorithm, StatusValues
from pydistsim.algorithm.node_wrapper import NodeAccess
from pydistsim.message import Message
from pydistsim.restrictions.communication import BidirectionalLinks
from pydistsim.restrictions.reliability import TotalReliability
from pydistsim.restrictions.topological import Connectivity
from pydistsim.restrictions.knowledge import InitialDistinctValues

from pydistsim import NetworkGenerator, Simulation
from pydistsim.logging import set_log_level, LogLevels, enable_logger, logger
from pydistsim.network.behavior import NetworkBehaviorModel

from pydistsim.gui import drawing as draw
%matplotlib inline
from matplotlib import pyplot as plt
from pydistsim.network import Node

import random
import math

set_log_level(LogLevels.INFO)
enable_logger()

from utils import *

## Parte 1: 

### 1. Implementación básica del problema
- Simule un conjunto de n generales (nodos) que deben decidir si atacar o retirarse.
- Permita que hasta f generales sean bizantinos, es decir, que puedan enviar mensajes contradictorios.
- Los generales leales deberán acordar una decisión común, cumpliendo las condiciones de consistencia y validez.


### 2. Protocolos de comunicación
- Implemente el protocolo recursivo propuesto por Lamport, Shostak y Pease (OM(f)).


In [ ]:
class ByzantineGenerals(NodeAlgorithm):
    def __init__(self, simulation, *args, f=None, verbose=False, **kwargs):
        super().__init__(simulation, *args, **kwargs)
        self.f = f
        self.siege = Siege(len(self.network.nodes()))
        if verbose:
            self.messages_counter = 0

    default_params = {
        "Observation" : "Observation",
        "Decision" : "Decision",

        "Value" : "Value",
    }

    class Status(StatusValues):
        COMMANDER = "COMMANDER"
        LIUTENANT = "LIUTENANT"
        AWAITING_ORDERS = "AWAITING_ORDERS"
        TRAITOR = "TRAITOR"
        ATTACK = "ATTACK"
        RETREAT = "RETREAT"

    S_init = (Status.COMMANDER, Status.LIUTENANT, Status.TRAITOR)
    S_term = (Status.ATTACK, Status.RETREAT)

    algorithm_restrictions = (
        # Restriccion no especificada: grafo completo
        BidirectionalLinks,
        Connectivity, 
        TotalReliability, # 3.A1 TODO no asumirlo?? ver notas
        InitialDistinctValues 
    )
        
    def initializer(self):
        self.apply_restrictions()
        traidores = random.sample(list(self.network.nodes()), self.f)
        commander_general = self.network.nodes_sorted()[0]
        for node in self.network.nodes():
            node.memory["commander"] = commander_general
            node.memory["decisions"] = {id : None for id in map(lambda n : n._internal_id, self.network.nodes())}
            node.memory["decision_threshold"] = define_general_threshold(node)
            node.memory["traitor"] = node in traidores
            node.memory["m"] = self.f
            node.status = self.Status.LIUTENANT
            node.push_to_inbox(Message(meta_header=NodeAlgorithm.INI, destination=node))
        commander_general.status = self.Status.COMMANDER



    @Status.LIUTENANT
    def spontaneously(self, node: NodeAccess, _: Message):
        node.memory["decisions"][node.memory["unique_value"]] = observe(node)
        if node.memory["traitor"]:
            TraitorActions.init(node, self)
        else:
            send_with_check(
                node, 
                datos=(node.memory["decisions"][node.memory["unique_value"]], node.memory["unique_value"]),
                dest=node.memory["commander"],
                msj_type=self.default_params["Observation"],
                algorithm=self
            )
        node.status = self.Status.AWAITING_ORDERS


    @Status.COMMANDER
    def spontaneously(self, node: NodeAccess, _: Message):
        logger.info("[{}] Is the commander general", f"General {node.memory['unique_value']}")
        node.memory["decisions"][node.memory["unique_value"]] = observe(node)
        if node.memory["traitor"]:
            TraitorActions.init(node, self)
    

    @Status.LIUTENANT
    def receiving(self, node: NodeAccess, message: Message):
        return self.error("LIUTENANT::receiving")


    @Status.COMMANDER
    def receiving(self, node: NodeAccess, message: Message):
        general_decision, general_id = message.data
        node.memory["decisions"][general_id] = general_decision
        if all(decision is not None for (id,decision) in node.memory["decisions"].items() if id != node.memory["unique_value"]):
            commander_decision = decide(node)
            if node.memory["traitor"]:
                TraitorActions.commander_decision(node, self)
            else:
                send_with_check(
                    node, 
                    datos=commander_decision,
                    dest=list(node.neighbors()),
                    msj_type=self.default_params["Decision"],
                    algorithm=self
                )
                if commander_decision == GeneralDecision.ATTACK:
                    self.siege.attack(node)
                    node.status = self.Status.ATTACK
                else:
                    self.siege.retreat(node)
                    node.status = self.Status.RETREAT

    @Status.AWAITING_ORDERS
    def receiving(self, node: NodeAccess, message: Message):
        commander_decision = message.data
        if node.memory["traitor"]:
            TraitorActions.respond_order(node, self) 
        elif commander_decision == GeneralDecision.RETREAT:
            self.siege.retreat(node)
            node.status = self.Status.RETREAT
        elif commander_decision == GeneralDecision.ATTACK:
            self.siege.attack(node)
            node.status = self.Status.ATTACK
        else:
            self.error("AWAITING_ORDERS::receiving")

    @Status.ATTACK
    def default(self, *args, **kwargs):    
        pass

    @Status.RETREAT
    def default(self, *args, **kwargs):    
        pass


- Muestre como crece la complejidad en mensajes según la cantidad de fallos f. Evalúe el caso f = 1 y luego f = 2.
- Mida el número total de mensajes intercambiados y el tiempo necesario para llegar al consenso.

In [8]:
f = 10
n = 10

net_gen = NetworkGenerator(directed=False)
net = net_gen.generate_complete_network(n)
sim = Simulation(net, check_restrictions=True)
sim.algorithms = ((ByzantineGenerals, {"f": f, "verbose" : True}),)


sim.run()
# fig = draw.draw_current_state(sim)
# fig

2025-11-21 16:44:36.578 | INFO     | pydistsim.simulation:__init__:59 - Simulation 0x175175312e0 created successfully.
2025-11-21 16:44:36.578 | INFO     | __main__:spontaneously:66 - [General 21] Is the commander general
2025-11-21 16:44:36.582 | INFO     | utils:observe:22 - [General 21] Observes they should attack
2025-11-21 16:44:36.583 | INFO     | utils:init:117 - [General 21] Is a traitor, so he sends the opposite decision
2025-11-21 16:44:36.584 | INFO     | utils:observe:22 - [General 22] Observes they should retreat
2025-11-21 16:44:36.585 | INFO     | utils:init:117 - [General 22] Is a traitor, so he sends the opposite decision
2025-11-21 16:44:36.586 | INFO     | utils:observe:22 - [General 23] Observes they should retreat
2025-11-21 16:44:36.586 | INFO     | utils:init:117 - [General 23] Is a traitor, so he sends the opposite decision
2025-11-21 16:44:36.587 | INFO     | utils:observe:22 - [General 24] Observes they should retreat
2025-11-21 16:44:36.588 | INFO     | utils

### 3. Análisis
- Determine experimentalmente el número mínimo de nodos necesario para alcanzar consenso frente a f fallos bizantinos.
- Verifique la condición teórica n ≥ 3f + 1.

## Otros

### Otros

In [ ]:
sim.reset()
plt.close()